# BitFit 实战
1.3B参数模型，全量微调显存25.80G，只微调bias即544768个参数，显存6.34G

## Step1 导入相关包

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "4"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = load_dataset("lifefabric/alpaca_data_cleaned.zh", split="train[:10000]")
ds

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10000
})

In [3]:
ds[:3]

{'instruction': ['给出三个保持健康的提示。', '三种主要颜色是什么?', '描述一个原子的结构。'],
 'input': ['', '', ''],
 'output': ['1 。 吃均衡和营养的饮食: 确保你的膳食包括各种水果和蔬菜, 瘦肉蛋白质, 全谷物和健康脂肪。 这有助于为您的身体提供最佳功能的必需营养素, 并有助于预防慢性疾病。\n\n2. 定期进行体育锻炼: 锻炼对于维持强壮的骨骼, 肌肉和心血管健康至关重要。 每周至少进行 150 分钟的中度有氧运动或 75 分钟的剧烈运动。\n\n3. 获得足够的睡眠: 获得足够的高质量睡眠对身心健康至关重要。 它有助于调节情绪, 改善认知功能, 并支持健康的生长和免疫功能。 每天晚上瞄准 7 - 9 小时的睡眠。',
  '三种主要颜色是红色, 蓝色和黄色。 这些颜色被称为主要颜色, 因为它们不能通过混合其他颜色来创建, 所有其他颜色都可以通过以各种比例组合来创建。 在添加色系中, 用于光线的主要颜色是红色, 绿色和蓝色 (R GB) 。',
  '原子是所有物质的基本构建块, 由三种类型的粒子组成: 质子, 中子和电子。 原子的结构可以被描述为中心的一个原子核, 周围是一团电子云。\n\n原子核由质子和中子组成。 质子是带正电的粒子, 中子是没有电荷的中性粒子。 这两个粒子都位于原子核中, 原子核位于原子的中心, 并包含原子的大部分质量。\n\n原子核周围是一团电子云。 电子是在原子核周围不断运动的带负电粒子。 电子云分为壳层或轨道, 每个壳层可以容纳一定数量的电子。 外壳中的电子数, 称为价壳, 决定了原子的化学性质。 \n\n在中性原子中, 原子核中的质子数等于电子云中的电子数, 因此正电荷和负电荷平衡, 原子没有总电荷。 质子的数量, 也称为原子序数, 决定了原子是什么元素。']}

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 三种主要颜色是什么?\n\nAssistant: 三种主要颜色是红色, 蓝色和黄色。 这些颜色被称为主要颜色, 因为它们不能通过混合其他颜色来创建, 所有其他颜色都可以通过以各种比例组合来创建。 在添加色系中, 用于光线的主要颜色是红色, 绿色和蓝色 (R GB) 。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'三种主要颜色是红色, 蓝色和黄色。 这些颜色被称为主要颜色, 因为它们不能通过混合其他颜色来创建, 所有其他颜色都可以通过以各种比例组合来创建。 在添加色系中, 用于光线的主要颜色是红色, 绿色和蓝色 (R GB) 。</s>'

## Step4 创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [10]:
sum(param.numel() for param in model.parameters())

1303111680

model size: 1.3B

model: 1.3G * 4 ~= 5.2G

gradient: 1.3G * 4 ~= 5.2G

optimizer: 1.3G * 4 * 2 ~= 10.4G

sum: 20.8G

## BitFit

In [11]:
# bitfit
# 选择模型参数里面的所有bias部分

num_param_update = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param_update += param.numel()

num_param_update

544768

In [12]:
num_param_update / sum(param.numel() for param in model.parameters())

0.000418051659240749

## Step5 配置训练参数

In [13]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [14]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [15]:
trainer.train()

Step,Training Loss
10,3.646100
20,3.296300
30,3.188500
40,3.000400
50,3.042300
60,3.037300
70,3.069400
80,3.015000
90,2.932300
100,2.836600


TrainOutput(global_step=1250, training_loss=2.7548036056518557, metrics={'train_runtime': 827.6969, 'train_samples_per_second': 12.082, 'train_steps_per_second': 1.51, 'total_flos': 9550997684305920.0, 'train_loss': 2.7548036056518557, 'epoch': 1.0})

In [18]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 以下是一些技巧:\n1) 进行多次练习。\n\n2) 把试卷中的某些答案或问题写在纸上或用铅笔标记或记下来。 \n3) 仔细阅读你所做的每个练习, 了解它是否包含你所遇到的所有问题？\n\n4) 仔细阅读每一个练习。 这需要花费时间, 甚至需要你反复思考和复现它。 但是, 这样做将给你以机会更好地了解它。'

## Step8 模型推理

In [15]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [18]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 使用技巧是利用一个数字来帮助你, 并可以自动生成正确答案。 这可以包括将问题分解为一个子题乘以答案的一半。 同时, 可以通过调整正确答案来提高命中率。 使用正确的答案可以提高正确率。'}]